## 2) Inferential analysis (matched cohort): baseline + exposures


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

project_root = Path.cwd().parent
data_path = project_root / "data" / "raw" / "HO_infxn_analysis.csv"
df = pd.read_csv(data_path)

df.shape


(120279, 78)

In [3]:
X.dtypes[X.dtypes == "object"]


age    object
dtype: object

In [4]:
df["age"].head(10)


0    74.5
1    71.1
2    60.4
3    60.4
4    43.4
5    43.4
6    56.5
7    56.5
8    63.1
9    63.1
Name: age, dtype: object

In [5]:
X_clean = X.copy()

# Convert age to numeric
X_clean["age"] = pd.to_numeric(X_clean["age"], errors="coerce")

# Verify no object columns remain
X_clean.dtypes[X_clean.dtypes == "object"]


Series([], dtype: object)

In [6]:
X2 = sm.add_constant(X_clean, has_constant="add")

# Keep only rows with complete predictors
mask = X2.notna().all(axis=1)
X2 = X2.loc[mask]
y2 = y.loc[mask]

print("Rows before:", len(y), "Rows after:", len(y2))

model_baseline = sm.Logit(y2, X2).fit(disp=False)
model_baseline.summary()


Rows before: 120279 Rows after: 112806


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           group_binary   No. Observations:               112806
Model:                          Logit   Df Residuals:                   112764
Method:                           MLE   Df Model:                           41
Date:                Fri, 13 Feb 2026   Pseudo R-squ.:                0.005595
Time:                        21:03:18   Log-Likelihood:                -70079.
converged:                       True   LL-Null:                       -70474.
Covariance Type:            nonrobust   LLR p-value:                4.556e-139
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.3460    8.1e+06  -4.27e-08      1.000   -1.59e+07    1.59e+07
const                   -0.3460   7.92e+06  -4.37e-08      1.000   -1.55e+07    1.55e+07
age                     -0.0023      0.000     -5.389      0.000      -0.003      -0.001
sex                      0.0047      0.014      0.351      0.726      -0.022       0.031
elix_index_mortality     0.0074      0.003      2.690      0.007       0.002       0.013
elix_AIDS               -0.2341      0.084     -2.799      0.005      -0.398      -0.070
elix_LUNG_CHRONIC       -0.0754      0.017     -4.489      0.000      -0.108      -0.042
elix_DEMENTIA           -0.1485      0.026     -5.623      0.000      -0.200      -0.097
elix_DEPRESS             0.0054      0.026      0.205      0.838      -0.046       0.057
elix_DIAB_UNCX          -0.0594      0.022     -2.688      0.007      -0.103      -0.016
elix_DIAB_CX            -0.0557      0.018     -3.120      0.002      -0.091      -0.021
elix_HTN_UNCX            0.0111      0.017      0.638      0.524      -0.023       0.045
elix_THYROID_HYPO       -0.0230      0.020     -1.170      0.242      -0.062       0.016
elix_THYROID_OTH         0.0624      0.047      1.332      0.183      -0.029       0.154
elix_CANCER_LYMPH        0.1136      0.045      2.502      0.012       0.025       0.203
elix_CANCER_LEUK         0.1914      0.061      3.143      0.002       0.072       0.311
elix_CANCER_METS         0.0491      0.066      0.746      0.455      -0.080       0.178
elix_OBESE               0.1425      0.026      5.528      0.000       0.092       0.193
elix_PERIVASC            0.1599      0.024      6.622      0.000       0.113       0.207
elix_CANCER_SOLID        0.1921      0.034      5.612      0.000       0.125       0.259
elix_CANCER_NSITU        0.6386      0.178      3.587      0.000       0.290       0.988
elix_ANEMDEF            -0.0025      0.019     -0.133      0.895      -0.040       0.035
elix_BLDLOSS             0.0612      0.067      0.916      0.360      -0.070       0.192
elix_COAG               -0.0597      0.043     -1.385      0.166      -0.144       0.025
elix_LIVER_SEV          -0.1427      0.060     -2.391      0.017      -0.260      -0.026
elix_NEURO_MOVT          0.0059      0.036      0.164      0.870      -0.065       0.076
elix_NEURO_SEIZ         -0.0921      0.030     -3.059      0.002      -0.151      -0.033
elix_PULMCIRC           -0.0212      0.033     -0.633      0.527      -0.087       0.044
elix_RENLFL_MOD         -0.0609      0.033     -1.865      0.062      -0.125       0.003
elix_ULCER_PEPTIC       -0.0698      0.058     -1.209      0.227      -0.183       0.043
elix_WGHTLOSS            0.1087      0.041      2.668      0.008       0.029       0.188
elix_DRUG_ABUSE         -0.1280      0.043     -3.006      0.003      -0.211      -0.045
elix_HTN_CX             -0.0314      0.022     -1.437      0.151      -0.074       0.011
elix_ALCOHOL             0.0154      

In [7]:
params = model_baseline.params
conf = model_baseline.conf_int()

or_table = pd.DataFrame({
    "OR": np.exp(params),
    "CI_low": np.exp(conf[0]),
    "CI_high": np.exp(conf[1]),
    "p_value": model_baseline.pvalues
}).sort_values("p_value")

or_table.head(10)


C:\Users\Alexander Jimenez\hospital-onset-infection-risk-model\.venv\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


,OR,CI_low,CI_high,p_value
elix_PERIVASC,1.173341,1.119119,1.230189,3.542495e-11
elix_DEMENTIA,0.861985,0.818501,0.907779,1.871937e-08
elix_CANCER_SOLID,1.211763,1.133145,1.295836,1.997779e-08
elix_OBESE,1.153140,1.096331,1.212894,3.239736e-08
age,0.997751,0.996934,0.998568,7.103987e-08
elix_LUNG_CHRONIC,0.927383,0.897355,0.958416,7.152595e-06
elix_CBVD,1.155197,1.081405,1.234024,1.838206e-05
elix_CANCER_NSITU,1.893865,1.335997,2.684679,3.344368e-04
elix_CANCER_LEUK,1.210903,1.074669,1.364407,1.675008e-03
elix_DIAB_CX,0.945850,0.913347,0.979510,1.806438e-03


In [9]:
cp_cols = [c for c in df.columns if c.endswith("_cp")]
print("CP cols:", cp_cols)

d = df.dropna(subset=["group_binary"]).copy()
X_cp = d[baseline_cols + cp_cols].copy()

for c in X_cp.columns:
    X_cp[c] = pd.to_numeric(X_cp[c], errors="coerce")

X_cp = sm.add_constant(X_cp, has_constant="add")

mask = X_cp.notna().all(axis=1)
print("Rows total:", len(X_cp))
print("Rows complete-case:", int(mask.sum()))
print("\nMissingness in CP cols (top):")
print(d[cp_cols].isna().mean().sort_values(ascending=False).head(10))


CP cols: ['DS_Entero_cp', 'ESBL_cp', 'CDiff_cp', 'VSE_cp', 'VRE_cp', 'MSSA_cp', 'MRSA_cp', 'DS_PsA_cp', 'DR_PsA_cp']
Rows total: 120279
Rows complete-case: 0

Missingness in CP cols (top):
DS_Entero_cp    0.0
ESBL_cp         0.0
CDiff_cp        0.0
VSE_cp          0.0
VRE_cp          0.0
MSSA_cp         0.0
MRSA_cp         0.0
DS_PsA_cp       0.0
DR_PsA_cp       0.0
dtype: float64


In [10]:
missing_rates = X_cp.isna().mean().sort_values(ascending=False)
missing_rates.head(15)


sex                     1.000000
age                     0.062114
const                   0.000000
elix_index_mortality    0.000000
elix_AIDS               0.000000
elix_LUNG_CHRONIC       0.000000
elix_DEMENTIA           0.000000
elix_DEPRESS            0.000000
elix_DIAB_UNCX          0.000000
elix_DIAB_CX            0.000000
elix_HTN_UNCX           0.000000
elix_THYROID_HYPO       0.000000
elix_THYROID_OTH        0.000000
elix_CANCER_LYMPH       0.000000
elix_CANCER_LEUK        0.000000
dtype: float64

In [11]:
baseline_cols_clean = []

# Keep age (acceptable missingness)
baseline_cols_clean.append("age")

# Elixhauser flags ONLY (exclude summary index)
baseline_cols_clean += [
    c for c in df.columns
    if c.startswith("elix_") and c != "elix_index_mortality"
]

len(baseline_cols_clean)


39

In [12]:
# Build design matrix
X_cp2 = d[baseline_cols_clean + cp_cols].copy()

# Force numeric
for c in X_cp2.columns:
    X_cp2[c] = pd.to_numeric(X_cp2[c], errors="coerce")

X_cp2 = sm.add_constant(X_cp2, has_constant="add")

mask2 = X_cp2.notna().all(axis=1)
print("Rows complete-case (fixed):", int(mask2.sum()))

X_cp2 = X_cp2.loc[mask2]
y_cp2 = y.loc[mask2]

model_cp = sm.Logit(y_cp2, X_cp2).fit(disp=False)
model_cp.summary()


Rows complete-case (fixed): 112808


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           group_binary   No. Observations:               112808
Model:                          Logit   Df Residuals:                   112759
Method:                           MLE   Df Model:                           48
Date:                Fri, 13 Feb 2026   Pseudo R-squ.:                0.007876
Time:                        21:14:20   Log-Likelihood:                -69920.
converged:                       True   LL-Null:                       -70475.
Covariance Type:            nonrobust   LLR p-value:                4.618e-201
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.6479      0.024    -26.687      0.000      -0.695      -0.600
age                  -0.0004      0.000     -0.821      0.411      -0.001       0.000
elix_AIDS            -0.2585      0.083     -3.116      0.002      -0.421      -0.096
elix_LUNG_CHRONIC    -0.0538      0.016     -3.434      0.001      -0.085      -0.023
elix_DEMENTIA        -0.0993      0.028     -3.546      0.000      -0.154      -0.044
elix_DEPRESS         -0.0392      0.016     -2.455      0.014      -0.071      -0.008
elix_DIAB_UNCX       -0.0601      0.022     -2.714      0.007      -0.103      -0.017
elix_DIAB_CX         -0.0615      0.019     -3.325      0.001      -0.098      -0.025
elix_HTN_UNCX         0.0158      0.017      0.902      0.367      -0.019       0.050
elix_THYROID_HYPO    -0.0474      0.018     -2.648      0.008      -0.082      -0.012
elix_THYROID_OTH      0.0027      0.041      0.066      0.947      -0.078       0.084
elix_CANCER_LYMPH     0.1365      0.043      3.163      0.002       0.052       0.221
elix_CANCER_LEUK      0.2366      0.055      4.277      0.000       0.128       0.345
elix_CANCER_METS      0.2154      0.023      9.441      0.000       0.171       0.260
elix_OBESE            0.1007      0.018      5.743      0.000       0.066       0.135
elix_PERIVASC         0.1713      0.022      7.743      0.000       0.128       0.215
elix_CANCER_SOLID     0.2635      0.021     12.261      0.000       0.221       0.306
elix_CANCER_NSITU     0.6155      0.178      3.450      0.001       0.266       0.965
elix_ANEMDEF         -0.0156      0.017     -0.925      0.355      -0.048       0.017
elix_BLDLOSS          0.0478      0.066      0.726      0.468      -0.081       0.177
elix_COAG             0.0434      0.024      1.842      0.065      -0.003       0.090
elix_LIVER_SEV        0.0015      0.044      0.034      0.973      -0.084       0.087
elix_NEURO_MOVT       0.0017      0.035      0.049      0.961      -0.068       0.071
elix_NEURO_SEIZ      -0.0729      0.029     -2.515      0.012      -0.130      -0.016
elix_PULMCIRC         0.0041      0.033      0.125      0.901      -0.060       0.069
elix_RENLFL_MOD      -0.0403      0.025     -1.597      0.110      -0.090       0.009
elix_ULCER_PEPTIC    -0.0543      0.058     -0.939      0.348      -0.168       0.059
elix_WGHTLOSS         0.1986      0.020     10.149      0.000       0.160       0.237
elix_DRUG_ABUSE      -0.1585      0.039     -4.055      0.000      -0.235      -0.082
elix_HTN_CX          -0.0168      0.027     -0.626      0.531      -0.070       0.036
elix_ALCOHOL          0.0254      0.031      0.829      0.407      -0.035       0.085
elix_AUTOIMMUNE      -0.0564      0.028     -2.012      0.044      -0.111      -0.001
elix_PSYCHOSES       -0.0117      0.028     -0.412      0.680      -0.067       0.044
elix_HF               0.0550      0.022      2.462      0.014       0.011       0.099
elix_RENLFL_SEV      -0.0557      0.032     -1.764      0.0

In [13]:
from scipy.stats import chi2

lr_stat = 2 * (model_cp.llf - model_baseline.llf)
df_diff = model_cp.df_model - model_baseline.df_model
p_lr = chi2.sf(lr_stat, df_diff)

lr_stat, df_diff, p_lr


(np.float64(318.4356176123547), 7.0, np.float64(6.963326561308847e-65))

In [14]:
params_cp = model_cp.params
conf_cp = model_cp.conf_int()

or_cp = pd.DataFrame({
    "OR": np.exp(params_cp),
    "CI_low": np.exp(conf_cp[0]),
    "CI_high": np.exp(conf_cp[1]),
    "p_value": model_cp.pvalues
})

or_cp_cpvars = (
    or_cp
    .loc[cp_cols]
    .sort_values("p_value")
)

or_cp_cpvars


,OR,CI_low,CI_high,p_value
DS_Entero_cp,0.988901,0.986663,0.991145,4.720887e-22
CDiff_cp,0.979412,0.969977,0.988939,2.532329e-05
VRE_cp,1.021928,1.011606,1.032356,2.819681e-05
ESBL_cp,0.987930,0.982267,0.993626,3.471782e-05
MSSA_cp,1.009267,1.003677,1.014888,1.132726e-03
MRSA_cp,0.990305,0.981623,0.999064,3.012960e-02
DR_PsA_cp,1.001324,0.989180,1.013616,8.317270e-01
VSE_cp,1.000486,0.994672,1.006334,8.701612e-01
DS_PsA_cp,1.000470,0.992000,1.009011,9.138002e-01
